# <font color=green>deepBreaks Applications</font>
## Modeling spectral tuning sites of opsin proteins based on amino-acid sequence...  

# <font color=red>Step 0: mySQL DB Setup -</font> Script 0
## *The following script sets up the schema for our vizphiz database.*
```

In [ ]:
#All neccessary packages to import for data process steps.
import mysql
import mysql.connector
#install mysql-connector-python // NOT mysql-connector
import re
import os
import datetime 
import subprocess

In [ ]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password= "Geass5566!!")

mycursor = mydb.cursor()

try:
  mycursor.execute("""
  DROP DATABASE vizphiz_db;
  """)
  mydb.commit() 
except:
  "vizphiz_db does not yet exist!"
  pass

mycursor.execute("""
CREATE DATABASE vizphiz_db;
""")
mydb.commit() 

mycursor.execute("""
USE vizphiz_db;
""")
mydb.commit() 


mycursor.execute("""
CREATE TABLE lamdamax
(
id int unsigned not null primary key,
genus varchar(50),
species varchar(50),
celltype varchar(50),
cellsubtype varchar(50),
lamdamax decimal(9,5),
error decimal(9,5),
chromophore varchar(50),
method varchar(50),
stage varchar(50),
refid int,
notes varchar(1000)
);
""")
mydb.commit() 


mycursor.execute("""
CREATE TABLE heterologous
(
hetid int unsigned not null primary key,
genus varchar(50),
species varchar(50),
accession varchar(500),
mutations varchar(500),
lamdamax decimal(9,5),
error decimal(9,5),
cellculture varchar(50),
purification varchar(50),
spectrum varchar(50),
sourcetype varchar(50),
refid int,
notes varchar(1000)
);
""")
mydb.commit() 


mycursor.execute("""
CREATE TABLE links
(
linkid int unsigned not null primary key,
accession varchar(500),
maxid int,
refid int,
evidence varchar(1000)
);
""")
mydb.commit() 


mycursor.execute("""
CREATE TABLE search
(
searchid int unsigned not null primary key,
researcher varchar(50),
month int,
year int,
engine varchar(500),
keywords varchar(500)
);
""")
mydb.commit() 


mycursor.execute("""
CREATE TABLE opsins
(
opsinid int unsigned not null primary key,
genefamily varchar(50),
genenames varchar(50),
genus varchar(50),
phylum varchar(25),
species varchar(50),
db varchar(50),
accession varchar(500),
dna varchar(10000),
aa varchar(3333),
refid int
);
""")

mydb.commit()

mycursor.execute("""
CREATE TABLE refs
(
refid int,
doilink varchar(100),
searchid int
);
""")
mydb.commit()

mydb.close()

In [ ]:
#All neccessary packages to import for data process steps.
import mysql
import mysql.connector
#install mysql-connector-python // NOT mysql-connector
import re
import os
import datetime 
import subprocess

# <font color=red>STEP 0: Data Base Setup -</font> Script 1 - Import heterologous.tsv into mySQL


In [ ]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  database= "vizphiz",
  password= "Geass5566!!"
)

#read file for data
file1 = open('heterologous.tsv', 'r', encoding="utf8")
Lines = file1.readlines()

count=0
for line in Lines:
    columns = line.split("\t")
    print(columns)
    mycursor = mydb.cursor()

    sql = "INSERT INTO vizphiz_db.heterologous (hetid, genus, species, accession, mutations, lamdamax, error, cellculture, purification, spectrum, sourcetype, refid) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    val = (columns[0], columns[1], columns[2], columns[3], columns[4], columns[5], columns[6], columns[7], columns[8], columns[9], columns[10], columns[11])
    print(sql)
    print(val)

    mycursor.execute(sql, val)

    mydb.commit()

    print(mycursor.rowcount, "record inserted.")

# <font color=red>STEP 0: Data Base Setup -</font> Script 2 - Import opsindb.tsv into mySQL

In [ ]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  database= "vizphiz",
  password= "Geass5566!!"
)

#read file for data
file1 = open('opsindb.tsv', 'r')
Lines = file1.readlines()

count=0
for line in Lines:
    columns = line.split("\t")

    mycursor = mydb.cursor()

    sql = "INSERT INTO vizphiz_db.opsins (opsinid, genefamily, genenames, phylum, genus, species, db, accession, dna, aa, refid) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    val = (columns[0], columns[1], columns[2], columns[3], columns[4], columns[5], columns[6], columns[7], columns[8], columns[9], columns[10])
    print(sql)
    print(val)

    mycursor.execute(sql, val)

    mydb.commit()

    print(mycursor.rowcount, "record inserted.")



# <font color=red>STEP 1: Extract Data From Vizphiz</font>
### Output = 5 different 'versions' // 'splits' of the data. 
### !Take outputs and run through MAFFT before moving on to STEP2!
### Suggested parameters for 'mafft' alignment are...
1. Fasta Format (Sorted)

2. Strategy = FFT-NS-2 // G-INS-1

In [2]:
#All neccessary packages to import for data process steps.
import mysql
import mysql.connector
#install mysql-connector-python // NOT mysql-connector
import re
import os
import datetime 
import subprocess

In [3]:
rod = re.compile('Rh[0-2]|exoRh')
d = re.compile('^NM_001014890.2$|^NM_001014890$')

In [5]:
#directory preperation
dt_label = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
seq_report_dir = str(f'vizphiz_data_splits_{dt_label}')
os.makedirs(seq_report_dir)

wd_output = f'{seq_report_dir}/wds.txt'
sws_output = f'{seq_report_dir}/swd.txt'
mws_output = f'{seq_report_dir}/mwd.txt'
rod_output = f'{seq_report_dir}/rod.txt'
wd_ni_output = f'{seq_report_dir}/wds_ni.txt'
inv_output = f'{seq_report_dir}/inv_only.txt'
nmoc_output = f'{seq_report_dir}/nmoc.txt'
mut_output = f'{seq_report_dir}/mut_only.txt'
wh_metadata = f'{seq_report_dir}/wds_meta.tsv'
sw_metadata = f'{seq_report_dir}/sws_meta.tsv'
mw_metadata = f'{seq_report_dir}/mws_meta.tsv'
rh_metadata = f'{seq_report_dir}/rod_meta.tsv'
wd_ni_metadata = f'{seq_report_dir}/wds_ni_meta.tsv'
inv_metadata = f'{seq_report_dir}/inv_meta.tsv'
nmoc_metadata = f'{seq_report_dir}/nmoc_meta.tsv'
mut_metadata = f'{seq_report_dir}/mut_meta.tsv'

meta_data_list = [wh_metadata,sw_metadata,mw_metadata,rh_metadata,wd_ni_metadata,inv_metadata,nmoc_metadata,mut_metadata]
meta_first_line = "\tLambda_Max\tSpecies\tOpsin_Family\tPhylum\tAccession\nBovine\t500.0000\tBos_tarus\tRh1\tChordata\tNM_001014890\n"
invert_first_line = "\tLambda_Max\tSpecies\tOpsin_Family\tPhylum\tAccession\nSquid\t473.0000\tTodarodes_pacificus\tRh1\tMollusca\tX70498\n"
bovine_seq = ">Bovine\nMNGTEGPNFYVPFSNKTGVVRSPFEAPQYYLAEPWQFSMLAAYMFLLIMLGFPINFLTLYVTVQHKKLRTPLNYILLNLAVADLFMVFGGFTTTLYTSLHGYFVFGPTGCNLEGFFATLGGEIALWSLVVLAIERYVVVCKPMSNFRFGENHAIMGVAFTWVMALACAAPPLVGWSRYIPEGMQCSCGIDYYTPHEETNNESFVIYMFVVHFIIPLIVIFFCYGQLVFTVKEAAAQQQESATTQKAEKEVTRMVIIMVIAFLICWLPYAGVAFYIFTHQGSDFGPIFMTIPAFFAKTSAVYNPVIYIMMNKQFRNCMVTTLCCGKNPLGDDEASTTVSKTETSQVAPA\n"
m = 0
s = 0
l = 0
r = 0
c = 0
z = 0
q = 0
mut = 0


In [6]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  database="vizphiz",
  password="Geass5566!!"
) 
mycursor = mydb.cursor()

sql = "select DISTINCT o.genus,o.species,o.genefamily,o.accession,h.lamdamax,o.aa,o.phylum,h.mutations from vizphiz_db.opsins o, vizphiz_db.heterologous h WHERE (o.accession = h.accession AND o.refid = h.refid); "
mycursor.execute(sql)
myresult = mycursor.fetchall()


for x in myresult:  

  if (x[4] == 0):
    pass   
  else:  
  #REG-DATA SECTION    
    with open(wd_output, 'a') as f:
      if m == 0:
          f.write(bovine_seq)
      if (d.match(x[3])):
        pass
      else:
        m += 1 
        #This makes the fasta format file
        seq = ">S" + str(m)
        f.write(seq)
        seq2 = str('\n' + x[5] + '\n')
        f.write(seq2)

    with open(wd_ni_output, 'a') as f:
      if x[6] != "Chordata" or d.match(x[3]):
        pass
      else:
        if c == 0:
          f.write(bovine_seq)
        c += 1 
        #This makes the fasta format file
        seq = ">S" + str(c)
        f.write(seq)
        seq2 = str('\n' + x[5] + '\n')
        f.write(seq2)

    with open(inv_output, 'a') as f:
      if (x[6] != "Chordata"):
        if q == 0:
          f.write(">Squid\nMGRDLRDNETWWYNPSIVVHPHWREFDQVPDAVYYSLGIFIGICGIIGCGGNGIVIYLFTKTKSLQTPANMFIINLAFSDFTFSLVNGFPLMTISCFLKKWIFGFAACKVYGFIGGIFGFMSIMTMAMISIDRYNVIGRPMAASKKMSHRRAFIMIIFVWLWSVLWAIGPIFGWGAYTLEGVLCNCSFDYISRDSTTRSNILCMFILGFFGPILIIFFCYFNIVMSVSNHEKEMAAMAKRLNAKELRKAQAGANAEMRLAKISIVIVSQFLLSWSPYAVVALLAQFGPLEWVTPYAAQLPVMFAKASAIHNPMIYSVSHPKFREAISQTFPWVLTCCQFDDKETEDDKDAETEIPAGESSDAAPSADAAQMKEMMAMMQKMQQQQAAYPPQGYAPPPQGYPPQGYPPQGYPPQGYPPQGYPPPPQGAPPQGAPPAAPPQGVDNQAYQA\n")
        q += 1 
        #This makes the fasta format file
        seq = ">S" + str(q)
        f.write(seq)
        seq2 = str('\n' + x[5] + '\n')
        f.write(seq2)
      else:
        pass

    with open(nmoc_output, 'a') as f:
      p = re.compile('_[G,A,L,M,F,W,K,Q,E,S,P,V,I,C,Y,H,R,N,D,T][0-9]+[G,A,L,M,F,W,K,Q,E,S,P,V,I,C,Y,H,R,N,D,T]')
      if p.search(x[3]) or "-" in x[3]:
        pass
      else:
        if z == 0:
          f.write(bovine_seq)
        if(d.match(x[3])):
          pass
        else:
          z += 1 
          #This makes the fasta format file
          seq = ">S" + str(z)
          f.write(seq)
          seq2 = str('\n' + x[5] + '\n')
          f.write(seq2)

    with open(mut_output, 'a') as f:
      p = re.compile('_[G,A,L,M,F,W,K,Q,E,S,P,V,I,C,Y,H,R,N,D,T][0-9]+[G,A,L,M,F,W,K,Q,E,S,P,V,I,C,Y,H,R,N,D,T]')
      if p.search(x[3]) or "-" in x[3]:
        if mut == 0:
          f.write(bovine_seq)
        if(d.match(x[3])):
          pass
        else:
          mut += 1 
          #This makes the fasta format file
          seq = ">M" + str(mut)
          f.write(seq)
          seq2 = str('\n' + x[5] + '\n')
          f.write(seq2)
      else:
        pass

    with open(sws_output, 'a') as f:
      p = re.compile('^SWS|^UVS')
      if p.match(x[2]) and x[6] == "Chordata":
        s+=1
        if s == 1:
          f.write(bovine_seq)
        #This makes the fasta format file
        seq = ">S" + str(s)
        f.write(seq)
        seq2 = str('\n' + x[5] + '\n')
        f.write(seq2)

    with open(mws_output, 'a') as f:
      p = re.compile('^MWS|^LWS')
      if p.match(x[2]) and x[6] == "Chordata":
        l+=1
        if l == 1:
          f.write(bovine_seq)
        #This makes the fasta format file
        seq = ">S" + str(l)
        f.write(seq)
        seq2 = str('\n' + x[5] + '\n')
        f.write(seq2)

    with open(rod_output, 'a') as f:
      p = re.compile('Rh[0-2]|exoRh')
      if p.match(x[2]):
        if r == 0:
          f.write(bovine_seq)
        if (x[6] != "Chordata") or d.match(x[3]):
          pass
        else:
          r+=1
          #This makes the fasta format file
          seq = ">S" + str(r)
          f.write(seq)
          seq2 = str('\n' + x[5] + '\n')
          f.write(seq2)

  #METADATA SECTION
    with open(wh_metadata, 'a') as g:
      if m == 1:
          g.write(meta_first_line)      
      if (d.match(x[3])):
        pass
      else:        
        md =  str("S" + str(m) + "\t" + str(x[4]).strip()) + "\t" + str(x[0]).strip().replace(' ','') + "_" + str(x[1]).strip().replace(' ','')  + "\t" + str(x[2]).strip() + "\t" + str(x[6]).strip().replace(' ','') + '\t' + x[3].strip() + "\n"
        g.write(md)

    with open(wd_ni_metadata, 'a') as g:
      p = re.compile('^Rtc|^BRh[0-3]|Pr[A-Z]|Rh1,Rh3|^IV|^inv')
      if x[6] != "Chordata" or d.match(x[3]):
        pass
      else:
        if c == 1:
          g.write(meta_first_line)      

        md =  str("S" + str(c) + "\t" + str(x[4]).strip()) + "\t" + str(x[0]).strip().replace(' ','') + "_" + str(x[1]).strip().replace(' ','')  + "\t" + str(x[2]).strip() + "\t" + str(x[6]).strip().replace(' ','') + '\t' + x[3].strip() + "\n"
        g.write(md)

    with open(inv_metadata, 'a') as g:
      if (x[6] != "Chordata"):
        if q == 1:
          g.write(invert_first_line)
        md =  str("S" + str(q) + "\t" + str(x[4]).strip()) + "\t" + str(x[0]).strip().replace(' ','') + "_" + str(x[1]).strip().replace(' ','')  + "\t" + str(x[2]).strip() + "\t" + str(x[6]).strip().replace(' ','') + '\t' + x[3].strip() + "\n"
        g.write(md)
      else:
        pass
          
    with open(sw_metadata, 'a') as g:
    #This makes the metadata formatted for a linear regression model.
      p = re.compile('^SWS|^UVS')
      if p.match(x[2]) and x[6] == "Chordata":
        if s == 1:
          g.write(meta_first_line)
        md =  str("S" + str(s) + "\t" + str(x[4]).strip()) + "\t" + str(x[0]).strip().replace(' ','') + "_" + str(x[1]).strip().replace(' ','')  + "\t" + str(x[2]).strip() + "\t" + str(x[6]).strip().replace(' ','') + '\t' + x[3].strip() + "\n"
        g.write(md)

    with open(mw_metadata, 'a') as g:
      #This makes the metadata formatted for a linear regression model.
      p = re.compile('^MWS|^LWS')
      if p.match(x[2]) and x[6] == "Chordata":
        if l == 1:
          g.write(meta_first_line)        
        md =  str("S" + str(l) + "\t" + str(x[4]).strip()) + "\t" + str(x[0]).strip().replace(' ','') + "_" + str(x[1]).strip().replace(' ','')  + "\t" + str(x[2]).strip() + "\t" + str(x[6]).strip().replace(' ','') + '\t' + x[3].strip() + "\n"
        g.write(md)

    with open(rh_metadata, 'a') as g:
      #This makes the metadata formatted for a linear regression model.
      p = re.compile('Rh[0-3]|exoRh')

      if p.match(x[2]):
        if r == 1:
          g.write(meta_first_line)      
        if (x[6] != "Chordata") or d.match(x[3]):
          pass
        else:  
          md =  str("S" + str(r) + "\t" + str(x[4]).strip()) + "\t" + str(x[0]).strip().replace(' ','') + "_" + str(x[1]).strip().replace(' ','')  + "\t" + str(x[2]).strip() + "\t" + str(x[6]).strip().replace(' ','') + '\t' + x[3].strip() + "\n"
          g.write(md)

    with open(nmoc_metadata, 'a') as g:
      p = re.compile('_[G,A,L,M,F,W,K,Q,E,S,P,V,I,C,Y,H,R,N,D,T][0-9]+[G,A,L,M,F,W,K,Q,E,S,P,V,I,C,Y,H,R,N,D,T]')
      if (p.search(x[3]) or "-" in x[3] or "_(" in x[3]):
        pass
      else:
        if z == 1:
          g.write(meta_first_line)      
        if(d.match(x[3])):
          pass
        else:        
          md =  str("S" + str(z) + "\t" + str(x[4]).strip()) + "\t" + str(x[0]).strip().replace(' ','') + "_" + str(x[1]).strip().replace(' ','')  + "\t" + str(x[2]).strip() + "\t" + str(x[6]).strip().replace(' ','') + '\t' + x[3].strip() + "\n"
          g.write(md)

    with open(mut_metadata, 'a') as g:
      p = re.compile('_[G,A,L,M,F,W,K,Q,E,S,P,V,I,C,Y,H,R,N,D,T][0-9]+[G,A,L,M,F,W,K,Q,E,S,P,V,I,C,Y,H,R,N,D,T]')
      if (p.search(x[3]) or "-" in x[3]):
        if mut == 1:
          g.write(meta_first_line)      
        if(d.match(x[3])):
          pass
        else:        
          md =  str("M" + str(mut) + "\t" + str(x[4]).strip()) + "\t" + str(x[0]).strip().replace(' ','') + "_" + str(x[1]).strip().replace(' ','')  + "\t" + str(x[2]).strip() + "\t" + str(x[6]).strip().replace(' ','') + '\t' + x[3].strip() + "\n"
          g.write(md)
      else:
        pass

# <font color=red>STEP 2: Align Raw Data and Format for 'deepBreaks'</font>
## REMINDER - You will need to change the directory for the 'mafft_exe' variable to the one of your own operating system!


In [ ]:
from Bio.Align.Applications import MafftCommandline
from Bio import AlignIO

data_split_list = [wd_output,sws_output,mws_output,rod_output,wd_ni_output,inv_output,nmoc_output,mut_output]
output_list = []
mafft_exe = 'C:/Users/safra/mafft-win/mafft.bat' 

for data in data_split_list:
    output = f'{data.split(".")[0]}_aligned.txt'
    mafft_cline = MafftCommandline(mafft_exe, input= f'./{data}')
    print(mafft_cline)

    stdout, stderr = mafft_cline()

    with open(output, "w") as handle:
        handle.write(stdout)
    align = AlignIO.read(output, "fasta")
    output_list.append(f'{output}')


In [ ]:
print(output_list)

for item in output_list:
    output = item.split('.')[0]
    print(output)

In [ ]:
#enter list of aligned text files here.
inputs = output_list
deep_breaks_input_data = []
# inputs = ['wds_aligned.txt','wds_ni_aligned.txt','rod_aligned.txt','nmoc_aligned.txt']
##enter list of names for desired formatted fasta files here.
# output = ['wds_fmt.fasta','wds_ni_fmt.fasta','rod_fmt.fasta','nmoc_fmt.fasta']
i=0
k = 0
for item in inputs:
    print(item)
    lines = open(inputs[i]).readlines()
    output = f'./{inputs[i].split(".")[0]}_db_fmt.fasta'
    deep_breaks_input_data.append(output)
    print(output)
    file = open(output, 'w')
    m=0
    for line in lines:
        snip = str(lines[k])
        if '>' in snip:
            if m == 0:
                m+=1
            else:
                file.write("\n")
            file.write(snip)
        else:
            entry = ""
            entry = str(snip.replace("\n",""))
            file.write(entry)
        k+=1
    k = 0
    i+=1
    file.close()

In [ ]:
print(deep_breaks_input_data)

# <font color=red>STEP 3: deepBreaks</font>
## THIS IS A LONG SECTION! 
### STEP 4 doesn't start until Cell 34

In [ ]:
# importing deepBreaks libraries 
from deepBreaks.utils import get_models, get_scores, get_params, make_pipeline
from deepBreaks.preprocessing import MisCare, ConstantCare, URareCare, CustomOneHotEncoder
from deepBreaks.preprocessing import FeatureSelection, CollinearCare
from deepBreaks.preprocessing import read_data
from deepBreaks.models import model_compare_cv, finalize_top, importance_from_pipe, mean_importance, summarize_results
from deepBreaks.visualization import plot_scatter, dp_plot, plot_imp_model, plot_imp_all
from deepBreaks.preprocessing import write_fasta
import numpy as np
import csv
import pandas as pd
import warnings
import datetime
import os
import shutil
import time

warnings.filterwarnings("ignore")
warnings.simplefilter('ignore')


In [ ]:
opsin_data_count = 0
ref_ali_list = []
ref_meta_list = []
ref_mod_list = []

In [ ]:
for opsindata in deep_breaks_input_data:
    # defining user params, file pathes, analysis type

    # path to sequences
    seqFileName = opsindata

    # path to metadata
    metaDataFileName = meta_data_list[opsin_data_count] 

    # name of the phenotype
    mt = 'Lambda_Max'

    # type of the sequences
    seq_type = 'aa'

    # type of the analysis if it is a classification model, then we put cl instead of reg
    ana_type = 'reg' 

    gap_threshold = 0.6

    # making a unique directory for saving the reports of the analysis
    print('direcory preparation')
    dt_label = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
    seqFile = seqFileName.split('/')[2]
    print(seqFile)
    seqFile = seqFile.split('.')[0]
    print(seqFile)
    report_dir = str(seqFile +'_' + mt + '_' + dt_label)
    os.makedirs(report_dir)


    print('reading meta-data')
    # importing metadata
    meta_data = read_data(metaDataFileName, seq_type = None, is_main=False)
    metaFile = metaDataFileName.split('/')[1]
    # importing sequences data
    print('reading fasta file')
    tr = read_data(seqFileName, seq_type = seq_type, is_main=True, gap_threshold=gap_threshold)

    #creating a copy of the input fasta file and metadata file into the report directory
    shutil.copy2(f'{seqFileName}' , report_dir)
    shutil.copy2(f'{metaDataFileName}' , report_dir)
    ref_meta_list.append(f'./{report_dir}/{metaFile}')
    #creating a copy of inout fasta file minus any of the sequences with no match in metadata file
    #this file will be used for aligning new test sequences by the user - using the mafft 'add sequence' function
    tr_for_ref = read_data(seqFileName, seq_type = seq_type, is_main=True, gap_threshold=1.0)
    tr_for_ref = tr_for_ref.merge(meta_data.loc[:, mt],  left_index=True, right_index=True)
    tr_for_ref.drop(mt, axis=1, inplace=True)
    fasta_file_name = f'{seqFile}_for_seq_insertion.fasta'
    write_fasta(dat = tr_for_ref, fasta_file = fasta_file_name , report_dir = report_dir)
    ref_ali_list.append(f'./{report_dir}/{fasta_file_name}')
    #converting the pandas df which contains all sequences after the gap_threshold has cleaned all the excess gaps
    write_fasta(dat = tr, fasta_file = f'{seqFile}_gap_dropped.fasta' , report_dir = report_dir)

    #making a copy of tr dataframe to isolate our msp invertebrate test sequences
    msp = tr.copy()
    #merging in lambda max values, simultaneously dropping all sequences without entries in metadata file
    tr = tr.merge(meta_data.loc[:, mt],  left_index=True, right_index=True)
    #tr.shape

    y = tr.loc[:, mt].values
    tr.drop(mt, axis=1, inplace=True)

    #copying the referece sequence into a dataframe later and saving a copy in the report directory 
    try:
        reference_seq = tr.loc['Bovine'].copy()
        ref_seq_name = 'bovine'
        #print(bovine)
    except:
        reference_seq = tr.loc['Squid'].copy()
        ref_seq_name = 'squid'
        #print(squid)
    reference_seq.to_csv(path_or_buf= f'{report_dir}/ref_sequence.csv',index = True,mode="w")

    #settingthe paramaters for our ML pipeline
    prep_pipeline = make_pipeline(
        steps=[
            ('mc', MisCare(missing_threshold=0.05)),
            ('cc', ConstantCare()),
            ('ur', URareCare(threshold=0.025)),
            ('cc2', ConstantCare()),
            ('one_hot', CustomOneHotEncoder()),
            ('feature_selection', FeatureSelection(model_type=ana_type, alpha=0.10, keep=False)),
            ('collinear_care', CollinearCare(dist_method='correlation', threshold=0.05, keep=False))
        ])

    #training models
    report, top = model_compare_cv(X=tr, y=y, preprocess_pipe=prep_pipeline,
                                models_dict=get_models(ana_type=ana_type),
                                scoring=get_scores(ana_type=ana_type),
                                report_dir=report_dir,
                                cv=12, ana_type=ana_type, cache_dir=report_dir)

                                
    time.sleep(1)
    #setting parameters for tuning the top 3 performing models
    prep_pipeline = make_pipeline(
        steps=[
            ('mc', MisCare(missing_threshold=0.05)),
            ('cc', ConstantCare()),
            ('ur', URareCare(threshold=0.025)),
            ('cc2', ConstantCare()),
            ('one_hot', CustomOneHotEncoder()),
            ('feature_selection', FeatureSelection(model_type=ana_type, alpha=0.10, keep=True)),
            ('collinear_care', CollinearCare(dist_method='correlation', threshold=0.05, keep=True))
        ])

    modified_top = []
    mtml = []
    for model in top:
        modified_top.append(make_pipeline(steps=[('prep', prep_pipeline), model.steps[-1]]))
        my_top_models = str(model[1:])
        #print(my_top_models)
        my_top_models = my_top_models.split("'")[3]
        mtml.append(my_top_models)
        #print(my_top_models)

    #print(mtml)
    time.sleep(1)

    #tuning the top 3 performing models 
    top = finalize_top(X=tr, y=y, top_models=modified_top, grid_param=get_params(),report_dir=report_dir, cv=10)
    #summarize the results by extracting feature importance and p-values and grouping correlated features.
    sr = summarize_results(top_models=top, report_dir=report_dir)
    #plot a scatter plot with -log of (p-value) column as the x-axis and the values of the other columns 
    scatter_plot = plot_scatter(summary_result=sr, report_dir=report_dir)

    time.sleep(1)

    #plot mean relative importance of each feature - corresponding to an amino acid position.
    mean_imp = mean_importance(top, report_dir=report_dir)

    dp_plot(importance=mean_imp,imp_col='mean', model_name='mean', report_dir=report_dir)
    tr = prep_pipeline[:4].fit_transform(tr)

    for model in top:
        model_name = model.steps[-1][0]
        dp_plot(importance=importance_from_pipe(model),
                imp_col='standard_value',
                model_name = model_name, report_dir=report_dir)
        
        plot_imp_model(importance=importance_from_pipe(model), 
                X_train=tr, y_train=y, model_name=model_name,
                    meta_var='meta', model_type=ana_type, report_dir=report_dir)

    time.sleep(1)
                    
    pl = plot_imp_all(final_models=top,
                    X_train=tr, y_train=y,
                    model_type = ana_type,
                    report_dir=report_dir, max_plots=10,
                    figsize=(2.5, 3))

    #translate candidate STSs to the bovine or squid equivalent 
    #bovine and squid sequence dataframes were saved earlier and are called again here
    m = 0
    tm = ''
    k=0
    gaps=0
    #import importance_report.csv from report_dir
    true_pos = []
    aa = []
    tmd = []
    df = pd.read_csv(f'{report_dir}\importance_report.csv')
    #take the list of important sites and translate them to the bovine standard equivalent, 
    #we do this by taking the site number and subtracting the number of '-' between the start of the sequence and the desired site. 
    for rows in reference_seq.values:  
        rows = str(rows)
        #print(rows)
        if rows == 'nan':
        #We want to write the 'true_pos', 'aa', and 'TMD' to the 'importance_report' csv file
            gaps += 1
            k += 1
            true_pos.append('NA')
            aa.append('-')
            tmd.append('NA')
        else:
            #print("The number of gaps is " + str(gaps))
            k+=1
            trans_site = k - gaps
            if ref_seq_name == 'bovine':
                if trans_site in range(3,37):
                    tm = 'N-Termina'
                elif trans_site in range(37,62):
                    tm = '1'
                elif trans_site in range(74,96):
                    tm = '2'
                elif trans_site in range(111,133):
                    tm = '3'
                elif trans_site in range(153,174):
                    tm = '4'
                elif trans_site in range(203,225):
                    tm = '5'
                elif trans_site in range(253,275):
                    tm = '6'
                elif trans_site in range(287,309):
                    tm = '7'
                else:
                    tm = 'CT/EC'
            else:
                if trans_site in range(3,34):
                    tm = 'N-Termina'
                elif trans_site in range(34,59):
                    tm = '1'
                elif trans_site in range(71,97):
                    tm = '2'
                elif trans_site in range(110,132):
                    tm = '3'
                elif trans_site in range(152,173):
                    tm = '4'
                elif trans_site in range(200,225):
                    tm = '5'
                elif trans_site in range(262,284):
                    tm = '6'
                elif trans_site in range(294,315):
                    tm = '7'
                else:
                    tm = 'CT/EC'                
            
            true_pos.append(str(trans_site))
            aa.append(rows)
            tmd.append(tm)
    true_pos.pop()
    aa.pop()
    tmd.pop()

    df['true_position'] = true_pos
    df['TMD'] = tmd
    df['amino_acid'] = aa
    df.to_csv(path_or_buf= os.path.join(report_dir,r'importance_report.csv'),index = 'Feature',mode="w")
    #df.head()

    #cycling to next set of data split files.
    opsin_data_count+=1


In [ ]:
with open(f'./{seq_report_dir}/path_to_ref_files.txt', 'w') as f:
    f.write('Here is a list of the paths to all aligned reference sequence files for inserting new sequences into an existing alignment before running a model prediction...\n')
    f.write(str(ref_ali_list) + '\n')
    f.write('Here is the list of paths for the corresponding metadata files...\n')
    f.write(str(ref_meta_list) + '\n')
    f.write('Finally, here is list of the paths to all the top performing models from each of the data splits...\n')
    f.write(str(ref_mod_list))

In [ ]:
#ref_ali_list = ['./wds_aligned_db_fmt_Lambda_Max_2023-08-08_15-08-05/wds_aligned_db_fmt.fasta', './swd_aligned_db_fmt_Lambda_Max_2023-08-08_15-10-55/swd_aligned_db_fmt_for_seq_insertion.fasta', './mwd_aligned_db_fmt_Lambda_Max_2023-08-08_15-13-55/mwd_aligned_db_fmt_for_seq_insertion.fasta', './rod_aligned_db_fmt_Lambda_Max_2023-08-08_15-15-29/rod_aligned_db_fmt_for_seq_insertion.fasta', './wds_ni_aligned_db_fmt_Lambda_Max_2023-08-08_15-18-19/wds_ni_aligned_db_fmt_for_seq_insertion.fasta', './inv_only_aligned_db_fmt_Lambda_Max_2023-08-08_15-23-06/inv_only_aligned_db_fmt_for_seq_insertion.fasta', './nmoc_aligned_db_fmt_Lambda_Max_2023-08-08_15-24-51/nmoc_aligned_db_fmt_for_seq_insertion.fasta']

#ref_meta_list = ['./wds_aligned_db_fmt_Lambda_Max_2023-08-08_15-08-05/wds_meta.tsv', './swd_aligned_db_fmt_Lambda_Max_2023-08-08_15-10-55/sws_meta.tsv', './mwd_aligned_db_fmt_Lambda_Max_2023-08-08_15-13-55/mws_meta.tsv', './rod_aligned_db_fmt_Lambda_Max_2023-08-08_15-15-29/rod_meta.tsv', './wds_ni_aligned_db_fmt_Lambda_Max_2023-08-08_15-18-19/wds_ni_meta.tsv', './inv_only_aligned_db_fmt_Lambda_Max_2023-08-08_15-23-06/inv_meta.tsv', './nmoc_aligned_db_fmt_Lambda_Max_2023-08-08_15-24-51/nmoc_meta.tsv']

#ref_mod_list = ['./wds_aligned_db_fmt_Lambda_Max_2023-08-08_15-08-05/lgbm.pkl', './swd_aligned_db_fmt_Lambda_Max_2023-08-08_15-10-55/rf.pkl', './mwd_aligned_db_fmt_Lambda_Max_2023-08-08_15-13-55/Lasso.pkl', './rod_aligned_db_fmt_Lambda_Max_2023-08-08_15-15-29/BayesianRidge.pkl', './wds_ni_aligned_db_fmt_Lambda_Max_2023-08-08_15-18-19/gbc.pkl', './inv_only_aligned_db_fmt_Lambda_Max_2023-08-08_15-23-06/LassoLars.pkl', './nmoc_aligned_db_fmt_Lambda_Max_2023-08-08_15-24-51/BayesianRidge.pkl']

In [ ]:
import os
import subprocess

i = 0
mafft_exe = 'C:/Users/safra/mafft-win/mafft.bat'
seq_add = 'mutant_seqs.fasta'

for ref_ali in ref_ali_list:
    try:
        mut_ali = f'{ref_ali[2:].split(".")[0]}_mutant_aligned.fasta'
        #print(mut_ali)
        cmd = [mafft_exe, '--add', seq_add, '--keeplength', ref_ali , '>', mut_ali ]
        aligner = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        out_put = aligner.communicate()[0].decode('utf8')
        
        #print(out_put)
        ref_copy = read_data( f'./{mut_ali}', seq_type = seq_type, is_main=True, gap_threshold=gap_threshold)
        mut_test = ref_copy
        meta_data = read_data(ref_meta_list[i], seq_type = None, is_main=False)

        ref_copy = ref_copy.merge(meta_data.loc[:, mt],  left_index=True, right_index=True)
        last_seq = ref_copy.shape[0]
        #print(ref_copy.shape)
        #print(last_seq)
        mut_test = mut_test.iloc[last_seq:].copy()
        #print(mut_test)

        load_mod = load_obj(ref_mod_list[i])
        #print(load_mod)
        predictions = load_mod.predict(mut_test)

        mut = pd.DataFrame(index=mut_test.index)
        mut['Prediction'] = predictions
        mut.to_csv(path_or_buf= f'{report_dir}/new_mutant_predictions.csv',mode="w")
    except:
        pass
    i+=1
